In [ ]:
# Import Packages

## system
import re
import sys
import traceback
import shutil
import codecs
import warnings 
warnings.filterwarnings ('ignore')

## Data structure
import pandas as pd
import numpy as np
import itertools
from decimal import Decimal
import datetime 
from datetime import datetime
import string
import ast
def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

## API related
import json
import requests
from pyquery import PyQuery as pq
# from selenium import webdriver 
# from selenium.webdriver import ChromeOptions

## NLP related
from cocoNLP.extractor import extractor
import jieba
import jieba.posseg as pseg

## Map/Plotting related
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 7]
import folium
print(folium.__version__)

## API URL & Credentials

### amap
url_amap = 'http://restapi.amap.com/v3/geocode/geo?parameters'
url_district='https://restapi.amap.com/v3/config/district?parameters'
KEY_amap = '460c7053ffe6fd4767354484f72f30df' 

### weibo
url_wb = 'https://m.weibo.cn/api/container/getIndex?' 
username_wb='wangjl_1995@163.com' # webdriver only
password_wb='wjl19950324'
App_Key:'1394316129' # Login only
App_Secret:'ababc16e7f47dba10d46bd4c7ae5c688'

## Define Lists for later uses

In [ ]:
xinguan_list=['感染', '阳性', '无症状', '确诊', '新冠', '病例', '奥密克戎', '德尔塔','密接','轨迹','行程','隔离', '封控']
word_filter=['核酸','全域','从业者','职员','某','卫健委','人员','工作人员','报告','冬奥','病例','区长','时间']
ncovtype_list=['奥密克戎','Omicron','德尔塔','Delta']

# Manually added locations 2022-01-26
# addr_xie=['丰台区万兴家居广场', '丰台区亿朋苑小区', '丰台区北水嘉伦水产品市场', '丰台区南三环西路兴源万柳综合市场', 
#           '丰台区右外中石化玉泉营加油站', '丰台区宜兰园小区', '丰台区宜兰园福宝隆生鲜超市', '丰台区槐房西路南亭新苑北区2号楼东二区7号', 
#           '丰台区玉泉营华兴饭庄', '丰台区玉泉营南三环西路50号花卉创意园', '丰台区玉泉营果品公司', '丰台区玉泉营果品冷库', 
#           '丰台区玉泉营水果市场丰台区万柳园小区', '丰台区石榴园西街234号北京世信海润食品有限公司', '丰台区纪家庙路188号', 
#           '丰台区草桥东路1号麦德龙超市', '丰台区草桥北区6号南城香', '丰台区草桥欣园2区京客隆超市', '丰台区郑王坟141号院西南郊冷库', 
#           '丰台区高立庄587号4号楼北鲜农农业科技发展有限公司', '丰台玉泉营果品冷库南站台五楼504库房', '房山区碧波园一果生鲜生活超市', 
#           '房山区碧波园世纪华联超市', '房山区大华制衣厂（长阳镇万兴路）', '房山区天骄骏园', '房山区百花蜂蜜厂（北广阳城大街8号）', 
#           '房山区碧波园', '房山区红日超市', '房山区长阳创业大厦', '房山区长阳绿地北京万民百汇大药房', '房山区长阳镇昊天北大街20号北亚骨科医院', 
#           '房山区长阳镇碧桂园小区2号楼底商', '房山区长阳镇碧波园底商2号楼怡仁康药房', '朝阳区东坝中路金泰丽富嘉园', '朝阳区京城皮肤医院（朝阳区）', 
#           '朝阳区平房乡石各庄一面一串', '朝阳区平房乡石各庄万家隆超市', '朝阳区平房乡石各庄北门', '朝阳区平房乡石各庄山东饺子', '朝阳区平房乡石各庄怡海花园', 
#           '朝阳区平房乡石各庄村313号家家福超市', '朝阳区平房乡石各庄村316号河南羊肉烩面', '朝阳区平房乡石各庄村378号正鸿五金', '朝阳区平房乡石各庄村378蔡超复印', 
#           '朝阳区平房乡石各庄王四营市场边批发市场', '朝阳区建外SOHO东区六号楼泰亭泰国料理', '朝阳区石各庄郭记川湘菜馆', '海淀区博雅西园社区', 
#           '海淀区工商银行(中关村南路支行)', '海淀区新技术大厦', '海淀区索真核酸检测点丹棱街海淀西街十字路口', '西城区中国邮政陶然亭邮政所', 
#           '西城区南礼士路56号北京儿童医院', '西城区德外大街16号北京市肛肠医院']

In [1315]:
### call AMAP district  API to get City-Districts-Subdistrcts 3 layers relationship, output as pd

def get_city_district(city_name):
    params_bj_distr={
        'keywords':city_name,
        'subdistrict':3,
        'key':KEY_amap
    }
    bj_distr_res=json.loads(requests.get(url_district, params_bj_distr).content)
    district_dict=bj_distr_res['districts'][0]['districts'][0]['districts']
    district_pd=pd.DataFrame.from_dict(district_dict)
    district_pd1=district_pd.explode('districts').reset_index(drop=True)
    district_pd1['name']=district_pd1['name'].apply(lambda x: re.sub(r'\区$', '',x))
    district_pd1['district_name']=district_pd1['districts'].apply(lambda x: x['name'])
    district_pd1['district_name_1']=district_pd1['districts'].apply(lambda x: re.sub(r'\镇$|\乡$|\街道$|\地区$', '',x['name']))
    district_pd1['district_center']=district_pd1['districts'].apply(lambda x: x['center'])
    bj_distr_pd=district_pd1[['adcode', 'name', 'center', 'district_name', 'district_name_1','district_center']]
    print(bj_distr_pd.columns)
    bj_distr_pd.to_excel(city_name+'_districts.xlsx', index=False)
    return bj_distr_pd
bj_distr_pd=get_city_district('北京市')

## Get Weibo Raw Data From Weibo Dev API

In [ ]:
# define weibo API headers

headers_wb ={
    'Host':'m.weibo.cn',
    'Referer': 'https://m.weibo.cn/u/1644948230',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'}
query_wb = {'type':'uid',
         'value':'1644948230'}

In [1317]:
bdf

,location_dtl,text,time,xinguan_dtl
0,"{'确诊病例1': ['果品冷库'], '确诊病例3': ['巴庄子路店', '现住址丰台区...",【#北京丰台新增4例感染者轨迹#发布】1月27日，#北京召开第277场疫情防控新闻发布会#。...,2022-01-27 16:36:24,"['感染', '阳性', '无症状', '确诊', '病例', '密接', '轨迹', '隔离']"
1,"{'发布会': ['北京市第277场新冠肺炎疫情防控工作新闻发布会'], '1': ['东城...",【#北京3人居家隔离期间擅自外出被拘# 核酸结果均为阴】#北京通报3起妨害疫情防控违法行为#...,2022-01-27 16:34:26,"['感染', '阳性', '无症状', '新冠', '病例', '隔离']"
2,"{'感染者70': ['丰台区王佐镇翡翠墅小区'], '感染者71': ['丰台区王佐镇翡翠...",【#北京新增5例本土感染者# 居住地涉及丰台、大兴】1月27日，#北京召开第277场疫情防控...,2022-01-27 16:14:57,"['感染', '阳性', '无症状', '确诊', '病例', '奥密克戎', '德尔塔',..."
3,"{'2': ['居家办公'], '3': ['区域内福利机构'], '4': ['区域内城市...",【#北京丰台部分区域提升管控措施#区域内居民足不出区，区域内单位员工原则上居家办公】#北京丰...,2022-01-27 00:09:51,['封控']
4,"{'发布会': ['北京市第276场新冠肺炎疫情防控工作新闻发布会'], '确诊病例1': ...",【#北京丰台新增感染者行程轨迹公布# 】1月26日，北京市第276场新冠肺炎疫情防控工作新闻...,2022-01-26 17:07:18,"['感染', '无症状', '确诊', '新冠', '病例', '密接', '轨迹', '行..."
...,...,...,...,...
75,"{'发布会': ['北京市第267场新冠肺炎疫情防控工作新闻发布会'], '1': ['VO...",【#北京海淀本土病例溯源情况发布# 病例曾接触加拿大国际邮件】1月17日，北京市第267场新...,2022-01-17 12:01:10,"['感染', '阳性', '确诊', '新冠', '病例', '奥密克戎', '轨迹']"
76,"{'发布会': ['北京市第267场新冠肺炎疫情防控工作新闻发布会'], '1': ['中国...",【#北京海淀病例曾接触来自加拿大国际邮件# 包装检测出奥密克戎】1月17日，北京市第267场...,2022-01-17 12:00:16,"['阳性', '确诊', '新冠', '病例', '奥密克戎']"
77,"{'发布会': ['北京市第267场新冠肺炎疫情防控工作新闻发布会'], '1': ['了本...",【北京海淀确诊病例目前已经判定密切接触者69人 均已落位管控】1月17日，北京市第267场新...,2022-01-17 11:56:27,"['感染', '阳性', '确诊', '新冠', '病例', '密接', '轨迹']"
78,"{'发布会': ['北京市第267场新冠肺炎疫情防控工作新闻发布会'], '1': ['48...",【自2022年1月22日零时起至3月底 抵京后需再测一次核酸 #抵京后未核酸检测可能会影响出...,2022-01-17 11:54:23,"['新冠', '奥密克戎']"


In [ ]:
# retrieve addresses with length>5 from scrapped raw text, outputs in dicts 

def address_from_text(text,length=5):
    regex_split   =  r'(\n)|(\+)|(\\)|(\.)|(\~)|(，)|(。)|(,)|(;)|(？)|(\?)|(、)|(；)|(（)|(）)|(：)|(【)|(】)|(#)|(\d+:\d+)'
    regex_address =  r'(地址)|(在)|(去)|(到)|(用.*餐)|(兼职)|(于)|(达)|(从)|(过)|(区.*号)|(超市)|(店)|(部)|(街)|(区)|(号)|(园)'
    regex_split_addr=r'(地址)|(住址)|(\d+个)|(\d日)|(\d时)|(第.*轮)|(核酸)|(感染)|(途中)|(住处)|(入户)|(北京市)|(风险)|(用.*餐)|(过)|(1月)|(乘)|(人员)|(单位)|(货物)|(定点)|(集中)|(隔离)|(观察)|(重点)|(病例)|(阳性)|(结果)|(清洁)|(记者)|(发布会)|(人群)|(全员)|(多次)|(症状)|(健康宝)|(物品)|(疫情)|(北京青年报)|(人民政府区长)|(人民政府副区长)|(通报)'
    wordtype_list=['a','b','bei','c','d','f','k','l','p','pba','r','t','u','ug','ude1','uj','v','vq','vn']
    keep_list=['支行','荟聚','东','西','南','北','生活','超市','例','移动','大','小','老','端']

    split_line = re.sub(regex_split ,' ',text.strip().lstrip().rstrip()).split()
#     print(split_line)
    addr_dict={'发布会':[],'1':[]}
    counter=0
    key=''
    for s in split_line:

        if re.findall(r'\d{1,2}$',s) and len(s)<=6:
            counter+=1
            key=s
            addr_dict[key]=[]
            
        elif re.findall(r'.*北京市第.*场新冠肺炎疫情.*新闻发布会.*',s):
            s=s.replace('召开','')
            addr_dict['发布会'].append(s)

        elif re.findall(regex_address, s) :
            if (any(int(num)>500 for num in re.findall('[0-9]+', s))) or any(x in s for x in ['政府','部长','区长']):
                pass
            
            else:
                # use jieba to remove words in wordtype, skip the words in keep_list
                words=pseg.cut(s)
                split_list=[w.word for w in words if w.flag in wordtype_list if w.word not in keep_list]                
                for ele in split_list:
                    s=s.replace(ele,' ')
                addr_pre =re.sub(regex_split_addr ,' ',s).split()

                # use pre-defined list to filter addresses
                for addr in addr_pre:
                    addr=addr.lstrip('北京').rstrip('购物')
                    if len(addr)>length and (addr.endswith('例')==False) :
                        if key=='':
                            addr_dict['1'].append(addr)
                        else:
                            addr_dict[key].append(addr)
                            
                # remove subsets for location lists 
                addr_dict['1']=[x for x in addr_dict['1'] if not any(x in y and x!=y for y in addr_dict['1'])]
                addr_dict[key]=[x for x in addr_dict[key] if not any(x in y and x!=y for y in addr_dict[key])]     
        
#     remove duplicates & sort for locations lists 
    addr_dict={k: sorted(list(set(v))) for k, v in addr_dict.items() if len(v)>0}
    
#     remove title-only results
    if len(addr_dict.keys())==1:
        addr_dict=None
    
    return addr_dict

In [1313]:
# read weibo raw data since start_date

def readHtml(start_date):

    query['containerid']='1076031644948230'
    query['since_id']=''
    info_list=[]
    info_col=[ 'mid', 'time','reposts_count', 'comments_count', 'attitudes_count','title','content1','text']    
    start_dt=datetime.strptime(start_date+' 00:00:00 +0800','%Y-%m-%d %H:%M:%S %z')
    start_str=datetime.strftime(start_dt, "%Y-%m-%d_%H:%M:%S")
    page = 1   
    while page:    

        url = url_wb + urlencode(query_wb)
        response = requests.get(url,headers=headers_wb,verify=False)
        
        if response.status_code == 200:
            index_json=response.json()
    #         print(index_json)
            cards = index_json['data']['cards']
            query['since_id']=index_json['data']['cardlistInfo']['since_id']
            cnt=1
            max_time=datetime.strptime(cards[0]['mblog']['created_at'],'%a %b %d %H:%M:%S %z %Y')

            if max_time>=start_dt:
                print('current maximum time: ',max_time, ' start reading......................')

                for card in cards:
                    try:
                        mblog=card['mblog']
                        if mblog['page_info']['type']=='search_topic':
                            mid = mblog['mid']
                            time=datetime.strptime(mblog['created_at'],'%a %b %d %H:%M:%S %z %Y')
                            reposts_count = mblog['reposts_count']
                            comments_count = mblog['comments_count']
                            attitudes_count = mblog['attitudes_count']
                            title=mblog['page_info']['page_title']
                            content1=mblog['page_info']['content1']
                            text = pq(mblog['text']).text()
                            isLongText = mblog['isLongText']
                            if isLongText:
                                text_id = mblog['id']
                                text_href = 'http://m.weibo.cn/statuses/extend?id=%s'%text_id
                                text_res = requests.get(text_href,headers=headers,verify=False)
                                text_json=text_res.json()
                                text = pq(text_json['data']['longTextContent']).text()

                            info = (mid, time, reposts_count, comments_count, attitudes_count,title,content1,text)
                            info_list.append(info)
                            print('page ',page,' card ',cnt,' inserted')

                        else:
                            print('page ',page,' card ',cnt,' omitted')

                        cnt+=1

                    except Exception as e:
                        traceback.print_exc()
                        continue

                print ('get page:%d'%page)
                page += 1

            else:
                print('current maximum time: ',max_time, ', time earlier than start date: ',start_dt,)
                break

    info_df = pd.DataFrame(info_list,columns=info_col)
    
    info_df['xinguan_flag']=  info_df['text'].apply(lambda x: any(ele in x for ele in xinguan_list))
    info_df['xinguan_dtl']=  info_df['text'].apply(lambda x: [ele for ele in xinguan_list if ele in x])
#     info_df['xinguan_type']=  info_df['text'].apply(lambda x: [ele for ele in ncovtype_list if ele in x])
    info_df['location_dtl']=  info_df['text'].apply(lambda x: address_from_text(x))
    info_df['location_flag'] =  info_df['location_dtl'].astype(bool)
    info_df['flag_count']= info_df['xinguan_flag'].astype(int) + info_df['location_flag'].astype(int) 
    
    info_df = info_df.astype(str)
    info_df['time']=info_df['time'].str[:19]
    
    ts=datetime.strftime(time.today(), "%Y-%m-%d_%H:%M:%S")
    info_df.to_excel('xinguan%s_%s_to_%s.xlsx'%(query['value'],start_str,ts), index=False)
    
    bdf=info_df[['location_dtl','text','time','xinguan_dtl']][(info_df.text.str.contains('北京'))&(info_df['flag_count']==2)].reset_index(drop=True)
    bdf.to_excel('loc_xinguan%s_%s_to_%s.xlsx'%(query['value'],start_str,ts), index=False)
    
    return bdf

In [1314]:
bdf=readHtml('2022-01-01')
print(bdf.columns)
print(bdf.shape)

current maximum time:  2022-01-28 10:14:15+08:00  start reading......................
page  1  card  1  omitted
page  1  card  2  omitted
page  1  card  3  inserted
page  1  card  4  omitted
page  1  card  5  omitted
page  1  card  6  omitted
page  1  card  7  omitted
page  1  card  8  omitted
page  1  card  9  omitted
page  1  card  10  omitted
get page:1
current maximum time:  2022-01-27 22:27:10+08:00  start reading......................
page  2  card  1  inserted
page  2  card  2  omitted
page  2  card  3  omitted
page  2  card  4  inserted
page  2  card  5  omitted
page  2  card  6  omitted
page  2  card  7  omitted
page  2  card  8  inserted
page  2  card  9  omitted
page  2  card  10  omitted
get page:2
current maximum time:  2022-01-27 18:28:00+08:00  start reading......................
page  3  card  1  omitted
page  3  card  2  inserted
page  3  card  3  inserted
page  3  card  4  omitted
page  3  card  5  omitted
page  3  card  6  omitted
page  3  card  7  inserted
page  3  

Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/1232520261.py", line 58, in readHtml
    text_json=text_res.json()
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


page  4  card  6  inserted
page  4  card  7  inserted
page  4  card  8  inserted
page  4  card  9  omitted
get page:4
current maximum time:  2022-01-27 15:49:52+08:00  start reading......................
page  5  card  1  omitted
page  5  card  2  omitted
page  5  card  3  omitted
page  5  card  4  inserted
page  5  card  5  omitted
page  5  card  6  omitted
page  5  card  7  omitted
page  5  card  8  inserted
page  5  card  9  omitted
page  5  card  10  omitted
get page:5
current maximum time:  2022-01-27 11:57:08+08:00  start reading......................
page  6  card  1  omitted
page  6  card  2  inserted


Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/1232520261.py", line 44, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


page  6  card  3  inserted
page  6  card  4  inserted
page  6  card  5  omitted
page  6  card  6  omitted
page  6  card  7  inserted
page  6  card  8  omitted
page  6  card  9  inserted
get page:6
current maximum time:  2022-01-27 08:48:17+08:00  start reading......................
page  7  card  1  inserted
page  7  card  2  omitted
page  7  card  3  omitted
page  7  card  4  inserted
page  7  card  5  omitted


...

Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/1232520261.py", line 58, in readHtml
    text_json=text_res.json()
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/jialuwang/opt/anaconda3/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


page  125  card  8  inserted
get page:125
current maximum time:  2022-01-05 16:47:07+08:00  start reading......................
page  126  card  1  omitted
page  126  card  2  omitted
page  126  card  3  inserted
page  126  card  4  omitted
page  126  card  5  omitted
page  126  card  6  inserted
page  126  card  7  omitted
page  126  card  8  omitted
page  126  card  9  inserted
get page:126
current maximum time:  2022-01-05 13:43:30+08:00  start reading......................
page  127  card  1  omitted
page  127  card  2  inserted


Traceback (most recent call last):
  File "/var/folders/rt/3d_k1c_126v7mb6b1schfk0r0000gn/T/ipykernel_68587/1232520261.py", line 44, in readHtml
    if mblog['page_info']['type']=='search_topic':
KeyError: 'page_info'


page  127  card  3  inserted
page  127  card  4  omitted
page  127  card  5  omitted
page  127  card  6  omitted
page  127  card  7  omitted
page  127  card  8  inserted
page  127  card  9  inserted
get page:127
current maximum time:  2022-01-05 10:32:50+08:00  start reading......................
page  128  card  1  inserted
page  128  card  2  omitted
page  128  card  3  omitted
page  128  card  4  omitted
page  128  card  5  omitted
page  128  card  6  inserted
page  128  card  7  omitted
page  128  card  8  omitted
page  128  card  9  inserted
page  128  card  10  omitted
get page:128
current maximum time:  2022-01-04 22:30:00+08:00  start reading......................
page  129  card  1  omitted
page  129  card  2  omitted
page  129  card  3  inserted
page  129  card  4  omitted
page  129  card  5  omitted
page  129  card  6  omitted
page  129  card  7  inserted
page  129  card  8  omitted
page  129  card  9  omitted
page  129  card  10  omitted
get page:129
current maximum time:  

KeyError: ''

# Process Weibo Raw Data into Mappable Data

In [1318]:
# Unstack location_dtl
def unstack_location_dtl(newpd):
    i=0
    newpd=pd.DataFrame(columns=['text','time','xinguan_dtl','key','original_addr','formatted_addr','location'])
    # newpd.columns=
    for i in range(bdf.shape[0]):
        loc_dict=ast.literal_eval(bdf['location_dtl'][i])
    #     print(loc_dict.keys())
        if '发布会' in loc_dict.keys():
            add_tag=loc_dict['发布会'][0]
            for key in loc_dict.keys():
                apd={}
                if key != '发布会':
                    apd['time']=bdf['time'][i]
                    apd['xinguan_dtl']=bdf['xinguan_dtl'][i][:-1]+','+add_tag+']'
                    apd['text']=bdf['text'][i]
                    apd['key']=key
                    apd['original_addr']=loc_dict[str(key)]
                    newpd=newpd.append(apd,ignore_index=True)
        else:
            for key in loc_dict.keys():
                apd={}
                apd['time']=bdf['time'][i]
                apd['xinguan_dtl']=bdf['xinguan_dtl'][i]
                apd['text']=bdf['text'][i]
                apd['key']=key
                apd['original_addr']=loc_dict[str(key)]            
                newpd=newpd.append(apd,ignore_index=True)

    newpd1=newpd.explode('original_addr').reset_index(drop=True)
    return newpd1

# AmapAPI return full address & LatLng
def call_amapAPI(address):
    url = 'http://restapi.amap.com/v3/geocode/geo?parameters'
    KEY = '460c7053ffe6fd4767354484f72f30df'   
    CITY = '北京市'
    ADDRESS = address
    params = {
        'city': CITY,
        'address': ADDRESS, 
        'key': KEY,
        'output': 'json'}
    j = json.loads(requests.get(url, params).content)
#     print(j)
    if j['status']=='0':
        pass
    elif j['status']=='1':
        item= j['geocodes'][0]
        adict={}
        adict['formatted_address']= item['formatted_address']
        adict['output_location']=[Decimal(item['location'].split(',')[1]),
                            Decimal(item['location'].split(',')[0])]
        return adict
    
# get full address & LatLng for unstacked data & filter out null address 
def get_addrLoc(newpd1):
    i=0 
    for i in range(newpd1.shape[0]):
        amap_geo=call_amapAPI(newpd1['original_addr'][i])
        if amap_geo:
            newpd1['formatted_addr'][i]=amap_geo['formatted_address']
            newpd1['location'][i]=amap_geo['output_location']
            
    geopd=newpd1[(newpd1.location.notnull())] 
    return geopd

In [1319]:
def final_geopd_formatting(geopd):
    
#     define final filters
    original_filter_f=['区域','工作','多家','德尔塔','部门','服务','小区内','报告',
                  '上市量','病毒','圈头','玉泉营便利店','快件','局长','多伦多',
                 '口岸','文件','奥密克戎','变异','肺炎','小时','监管','一次','社区居委会'
                  '疾控中心','东坝家属区','左右','于燕京','胸部影像学',
                 '员工','买菜','四个区','机场','全域']

    formatted_filter_f=['北京市东城区','北京市西城区','北京市朝阳区','北京市丰台区','北京市石景山区',
                        '北京市海淀区','北京市门头沟区','北京市房山区','北京市通州区','北京市顺义区',
                        '北京市昌平区','北京市大兴区','北京市怀柔区','北京市平谷区','北京市密云区',
                        '北京市延庆区','北京市',
                        '北京市海淀区阳光','北京市朝阳区北京国际俱乐部','北京市通州区通州','北京市西城区北大人民医院',
                        '北京市大兴区社区居委会','北京市大兴区南城','北京市大兴区南院','北京市大兴区居住地','北京市大兴区阳光',
                        '北京市房山区房山','北京市房山区疾控中心','北京市房山区罗森便利店' ]

    zone_filter_f=['门头','石景','平谷','昌平','东城','延庆','怀柔']
    
    print(geopd.shape,'before final filters')
    
#     apply final filters
    original_filter_mask = geopd['original_addr'].apply(lambda x: any(ele in x for ele in original_filter_f)) 
    geopd['zone']=geopd['formatted_addr'].str[3:5]
    geopd=geopd[(~original_filter_mask) & \
                (~geopd['zone'].isin(zone_filter_f))& \
                (~geopd['formatted_addr'].isin(formatted_filter_f))]

    print(geopd.shape,'after final filters')

    # add date & color
    geopd['daydiff']=(pd.to_datetime("now")-pd.to_datetime(geopd['time'],format='%Y-%m-%d %H:%M:%S'))/np.timedelta64(1,'D')
    i=0
    for i in range(geopd.shape[0]):
        if  geopd['daydiff'][i]<=1:
            geopd['color'][i] ='gray'
        elif  geopd['daydiff'][i]<=2:
             geopd['color'][i] ='purple'
        elif  geopd['daydiff'][i]<=4:
             geopd['color'][i] ='red'
        elif  geopd['daydiff'][i]<=7:
             geopd['color'][i] ='orange'
        elif  geopd['daydiff'][i]<=10:
             geopd['color'][i] ='blue'
        elif  geopd['daydiff'][i]<=14:
             geopd['color'][i] ='pink'
        else :
             geopd['color'][i] ='green' 

    # drop duplicates
    geopd=geopd.drop_duplicates(subset='formatted_addr').sort_values(by='formatted_addr').reset_index(drop=True)

    # drop subsets
    i=0
    alist=[]
    for i in range(geopd.shape[0]):
        j=i+1
        for j in range(geopd.shape[0]):
            if (geopd['formatted_addr'][i] in geopd['formatted_addr'][j]):
                if (len(geopd['formatted_addr'][i])< len(geopd['formatted_addr'][j])):
                    alist.append(i) 
                    
    geopd = geopd.drop(index=alist).reset_index(drop=True)

    print(geopd.shape,'after drop duplicates & subsets')
    return geopd_final
    # i=0
    # for i in range(geopd.shape[0]):
    # #     for j in range(geopd.shape[0]):
    #     print('--------------',i,'--------------')
    #     print(geopd['formatted_addr'][i],geopd['original_addr'][i])

In [1321]:
newpd1=unstack_location_dtl(bdf)
geopd=get_addrLoc(newpd1)
geopd_final=final_geopd_formatting(geopd)

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区果品冷库', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '丰台区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110106', 'street': [], 'number': [], 'location': '116.342071,39.851300', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区巴庄子路', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '丰台区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110106', 'street': [], 'number': [], 'location': '116.321282,39.828397', 'level': '道路'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区巴庄子村', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '丰台

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区万柳园小区|38号楼', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '丰台区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110106', 'street': [], 'number': [], 'location': '116.333957,39.851890', 'level': '门牌号'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区南郊冷库', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '丰台区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110106', 'street': [], 'number': [], 'location': '116.339989,39.850963', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市丰台区亿朋苑一区|12', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'dis

{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区白纸坊社区卫生服务中心', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.363322,39.877456', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区内西街甲10号院', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.360687,39.871787', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infoco

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区鸭子桥路1号院', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.346435,39.876739', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区广安门', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.343178,39.882633', 'level': '热点商圈'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市朝阳区安贞里三区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district'

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市朝阳区安贞里三区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '朝阳区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110105', 'street': [], 'number': [], 'location': '116.403953,39.969881', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区鸭子桥路1号院', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.346435,39.876739', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区广安门', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district':

...

{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市顺义区市场', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '顺义区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110113', 'street': [], 'number': [], 'location': '116.625566,40.129295', 'level': '兴趣点'}]}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市海淀区信息路2号', 'country': '中国', 'province': '北京市', 

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市海淀区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '海淀区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110108', 'street': [], 'number': [], 'location': '116.297700,39.959893', 'level': '区县'}]}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市朝阳区多伦多', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '朝阳区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110105', 'street': [], 'number': [], 'location': '116.459196,39.917111

{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市海淀区公司', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '海淀区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110108', 'street': [], 'number': [], 'location': '116.317199,39.983553', 'level': '兴趣点'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市海淀区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '海淀区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110108', 'street': [], 'number': [], 'location': '116.297700,39.959893', 'level': '区县'}]}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'cou

{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市西城区疾控中心', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '西城区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110102', 'street': [], 'number': [], 'location': '116.395190,39.881041', 'level': '兴趣点'}]}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '0', 'info': 'ENGINE_RESPONSE_DATA_ERROR', 'infocode': '30001'}
{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市海淀区', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '海淀区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adc

KeyError: 'color'

## Folium Map Plotting

In [ ]:
# define the city map
f = folium.Figure(width=1000, height=800)
city_map = folium.Map(tiles='https://webrd02.is.autonavi.com/appmaptile?lang=zh_en&size=1&scale=1&style=8&x={x}&y={y}&z={z}',
                      attr='高德-中英文对照',location=[39.91, 116.35], zoom_start=11,min_zoom=10,
                      max_zoom=18,control_scale=True,no_touch=False).add_to(f)

# define add ncovMarkets
def add_ncovMarker(amap,pd):
    amap=city_map
    geopd=pd
    i=0
    for i in range(geopd.shape[0]):
        try:
            message='''
            <html>
            <body>
            <font size="2"> 
            <b>Time : </b>{} <br/>
            <b>Tag : </b>{} <br> 
            <b>Key : </b>{} <br> 
            <b>Source : </b>{}
            </font>
            </body
            </html>
            '''.format(geopd['time'][i],geopd['xinguan_dtl'][i],geopd['key'][i],geopd['text'][i])
        except:
            message=''

        iframe = folium.IFrame(html=message, width=250, height=100)
        try:
            folium.Marker(location =geopd['location'][i],
                          popup=folium.Popup(iframe), 
                          tooltip=parse_zhch(geopd['formatted_addr'][i]),
                          icon=folium.Icon(color=geopd['color'][i]
                                           ,prefix='fa',icon='exclamation-triangle')
                         ).add_to(city_map)
        except:
            print(geopd['formatted_addr'][i], 'omitted-------------')
            pass
    return city_map

cmap=add_ncovMarker(amap,geopd_final)
cmap

In [1233]:

# geopd=newpd1[(newpd1.location.notnull())&\
#             (~newpd1['formatted_addr'].isin(new_d))& 
#             (newpd1.original_addr.str.len()>=5)]
# #              (newpd1.original_addr.str.len()>=5)&
# #             
            
# print(geopd.shape)

# final_filter=['区域','工作','多家','德尔塔','部门','服务','小区内','报告',
#               '上市量','病毒','圈头','玉泉营便利店','快件','局长','多伦多',
#              '口岸','文件','奥密克戎','变异','肺炎','小时','监管','一次',
#               '疾控中心','东坝家属区','左右','北京市房山区房山','于燕京','胸部影像学',
#              '员工']

# mask = geopd['original_addr'].apply(lambda x: any(ele in x for ele in final_filter)) 
# geopd['zone']=geopd['formatted_addr'].str[3:5]
# # geopd
# geopd=geopd[(~mask) & (~geopd['zone'].isin(['门头','石景','平谷','昌平']))]
# geopd=geopd.drop_duplicates(subset='formatted_addr').reset_index(drop=True)
# i=0
# for i in range(geopd.shape[0]):
#     print('--------------',i,'--------------')
#     print(geopd['formatted_addr'][i],geopd['original_addr'][i])

(362, 7)
-------------- 0 --------------
北京市丰台区巴庄子路 巴庄子路店
-------------- 1 --------------
北京市丰台区巴庄子村 现住址丰台区玉泉营街道巴庄子村
-------------- 2 --------------
北京市丰台区万柳园小区 丰台万柳园小区
-------------- 3 --------------
北京市丰台区南郊冷库 西南郊冷库
-------------- 4 --------------
北京市丰台区王佐镇万源小区 丰台区王佐镇翡翠墅小区
-------------- 5 --------------
北京市大兴区祥龙大院 大兴区旧宫镇庑殿路祥龙大院
-------------- 6 --------------
北京市丰台区石榴园西街|234号 石榴园西街234号
-------------- 7 --------------
北京市丰台区万柳园小区|38号楼 现住址万柳园小区38号楼2单元
-------------- 8 --------------
北京市丰台区亿朋苑一区|12 现住址亿朋苑一区12号楼3单元
-------------- 9 --------------
北京市丰台区亿朋苑一区|6 现住址亿朋苑一区6号楼5门
-------------- 10 --------------
北京市丰台区万芳园一区|10号楼 现住址万芳园一区10号楼1单元
-------------- 11 --------------
北京市大兴区南院 丰公寓南院31号平房
-------------- 12 --------------
北京市丰台区云岭商务楼 丰台区南四环云岭商务楼
-------------- 13 --------------
北京市丰台区怡海花园店 怡海花园店
-------------- 14 --------------
北京市丰台区恒富中街4号院|2号楼 恒富中街4号院2号楼5单元
-------------- 15 --------------
北京市怀柔区南房村 C座17号南房
-------------- 16 --------------
北京市丰台区万佳利大院 万佳利大院平房区
-------------- 17 -----

In [ ]:
# #禁止图片和css加载
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.managed_default_content_settings.images": 2}
# chrome_options.add_experimental_option("prefs", prefs)

# option = ChromeOptions()
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# # 如果想加载图片，就把下面第二句话改第一句话，删掉上面的“禁止图片和css加载”部分
# # wb = Chrome(options=option)
# driver = webdriver.Chrome("/Users/jialuwang/Downloads/chromedriver",
#                           options=chrome_options) 




# # LoginWeibo

# def loginWeibo(username, password): 
#     driver.get('https://passport.weibo.cn/signin/login') 
#     time.sleep(1) 
#     driver.find_element_by_id("loginName").send_keys(username) 
#     driver.find_element_by_id("loginPassword").send_keys(password) 
#     time.sleep(1) 
#     driver.find_element_by_id("loginAction").click() 
#     time.sleep(2)
#     driver.find_element_by_id('messageCheck').click()
#     time.sleep(10)
#     driver.find_element_by_id('message_sms_login').click()
#     yanzheng=input("输入六位验证码：")
#     for i in range(0,6):
#         driver.find_element_by_xpath('//div[@class="num clearfix"]/input[{}]'.format(i+1)).send_keys("yanzheng[i]")
#     time.sleep(5)
#     cookies = driver.get_cookies()
#     cookie = [item["name"] + "=" + item["value"] for item in cookies]
#     cookiestr = ';'.join(item for item in cookie)
#     return cookiestr
#     #driver.close() 
# driveLogIn = loginWeibo(username_wb, password_wb)
# driveLogIn

In [ ]:
# # TESTING------------
# def get_page():
#     query = {
#         'type':'uid',
#         'value':'2830678474',
#         'containerid':'1076031644948230',
#         'since_id':''
#     }
#     url = url_wb + urlencode(query)
#     try:
#         response = requests.get(url,headers=headers,verify=False)
#         if response.status_code == 200:
#             return response.json()
#     except requests.ConnectionError as e:
#             print('Get First Page Error',e.args)
            
# wb_json=get_page()


# i=1
# info_list=[]
# for card in wb_json['data']['cards']:
#     mblog=card['mblog']
#     mid = mblog['mid']
#     time=mblog['created_at']
#     reposts_count = mblog['reposts_count']
#     comments_count = mblog['comments_count']
#     attitudes_count = mblog['attitudes_count']
#     title=mblog['page_info']['page_title']
#     content1=mblog['page_info']['content1']
    
#     try:
#         content2=mblog['page_info']['content2']
#     except:
#         content2=''
        
#     isLongText = mblog['isLongText']
    
#     text = pq(mblog['text']).text()

#     if isLongText:
#         print('looong')
#         text_id = mblog['id']
#         text_href = 'http://m.weibo.cn/statuses/extend?id=%s'%text_id
#         text_res = requests.get(text_href,headers=headers,verify=False)
#         text_json=text_res.json()
#         text = pq(text_json['data']['longTextContent']).text()
# #         print(pq(text).text())          
#     info = (mid, time, reposts_count, comments_count, attitudes_count,title,content1,content2,text)
#     info_list.append(info)
# #     print(card)
#     print('-----------------',i,'-------------------')
#     i+=1

In [ ]:
# import cpca
# # location_str=bdf['text']
# # location_str=location_str.replace("\n",'')
# add_df=pd.DataFrame()
# for loc in bdf['text']:
#     try:
#         cdf = cpca.transform_text_with_addrs(loc)
#         cdf = cdf[(cdf['省']=='北京市')&(cdf['市'].notnull())]
#     except Exception as e:
# #         traceback.print_exc()
#         continue
#     add_df=add_df.append(cdf)
    
# add_df
    

In [ ]:
# ex = extractor()

# def location_from_text(text):
#     A=[]
#     for loc in ex.extract_locations(text):
#         loc=loc.replace('北京市','').replace('北京','').replace('\n','')
#         if (len(loc)>=5) and (any(ele in loc for ele in location_list)==True) \
#         and (any(ele in loc for ele in word_filter)==False) and (loc not in A) : 
#             for district in district_list:
#                 if district in loc and '区' not in loc:
#                     loc=loc.replace(district,district+'区')
#                     A.append(loc)
#     return [x for x in A if not any(x in y and x!=y for y in A)]

In [ ]:

# def get_containerid():
#     query = {'type':'uid',
#          'value':'1644948230'}
#     url = url_wb + urlencode(query)
#     try:
#         response = requests.get(url,headers=headers)
        
#         if response.status_code == 200:
#             containerid_dict={}
#             wb_fp=response.json()
            
#             for item in wb_fp['data']['tabsInfo']['tabs']:
#                 containerid_dict[item['tabKey']]=item['containerid']
#             return containerid_dict
        
#     except requests.ConnectionError as e:
#         print('Get First Page Error',e.args)